In [1]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


from nltk.corpus import stopwords

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup

from datetime import date
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('punkt')
from collections import defaultdict
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hydro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Preprocessing

In [2]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_xml(text):
    return re.sub(r'<[^<]+?>', '', text)

def remove_newlines(text):
    return text.replace('\n', ' ') 
    

def remove_manyspaces(text):
    return re.sub(r'\s+', ' ', text)

def clean_text(text):
    text = remove_xml(text)
    text = remove_newlines(text)
    text = remove_manyspaces(text)
    return text

# lemmatizatioin using Spacy to count the appearance of each word
def space (comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])


def join_edited_string(edited_tweets):
          
    return edited_tweets.str.cat(sep=' ')


In [3]:
def analysis_to_csv(data_path, output_path):

    days=os.listdir(data_path)
    days =[ i for i in days if i.split('.')[0].split('-')[-1]=='en']
    data_list = []
    for i in days:
        file_path  = i
        path = os.path.join(data_path, file_path)
        df_day = pd.read_csv(path,lineterminator='\n')
        print(df_day)
        data_list.append(df_day)
    df = pd.concat(data_list)
    df['day'] = df['created_at'].apply(lambda x: x.split(' ')[0])
    df['hour'] = df['created_at'].apply(lambda x: x.split(' ')[1])
    df['hour'] = df['hour'].apply(lambda x: x.split('-')[0])
    df = df.reset_index(drop=True)
    day_count = df.groupby('day').count()['text']
    nlp = spacy.load('en_core_web_sm')
    df['text'] = df['text'].apply(lambda x: remove_emoji(x))
    df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    df['text'] = df['text'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
    df['text'] = df['text'].apply(lambda x: re.sub('\[[^]]*\]', '', x))
    df['text'] = df['text'].apply(lambda x: re.sub(r'http\S+', '', x))
    df['text'] = df['text'].str.replace('[^\w\s]','')
    import nltk
    nltk.download('stopwords')
    stop = set(stopwords.words('english'))
    punctuation = list(string.punctuation)
    stop.update(punctuation)
    df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    df['text']= df['text'].apply(clean_text)
    df_02 = df.copy()
    nlp = spacy.load('en_core_web_sm')
    df_02['text']= df_02['text'].apply(space)
    joined_string = join_edited_string(df['text'])
    tokens = joined_string.split(' ')
    joined_string1 = join_edited_string(df_02['text'])
    tokens1 = joined_string1.split(' ')
    unique_words = set(tokens)
    unique_words1 = set(tokens1)
    unique_words.update(unique_words1)
    word_to_ind = dict((word, i) for i, word in enumerate(unique_words))
    ind_to_word = dict((i, word) for i, word in enumerate(unique_words))
    ncr = pd.read_excel("D:/data2/NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx")
    ncr = ncr[['English (en)', 'Positive', 'Negative', 'Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust']]
    emotions = ['Anger', 'Anticipation','Disgust','Fear', 'Joy','Sadness', 'Surprise', 'Trust']
    df_tweets = df.copy(deep=True)
    emo_info = {'emotion':'' , 'emo_frq': defaultdict(int) }
    list_emotion_counts = []
    for emotion in emotions:
            emo_info = {}
            emo_info['emotion'] = emotion
            emo_info['emo_frq'] = defaultdict(int)
            list_emotion_counts.append(emo_info)
    nrow = df_tweets.shape[0]
    df_emotions = pd.DataFrame(0, index=df_tweets.index, columns=emotions)
    process_count=0
    for i in range(nrow):
        
            process_count+=1
            if process_count % 100==0:
                print(process_count)
                print(df_tweets.loc[i])
            tweet = word_tokenize(df_tweets.loc[i]['text'])
            for word in tweet:
                word_stemmed = word
                result = ncr[ncr['English (en)'] == word_stemmed]
                if not result.empty:
                    # update the tweet-emotions counts
                    for idx, emotion in enumerate(emotions):
                        df_emotions.at[i, emotion] += result[emotion].iloc[0]
                        # update the frequencies dictionary list
                        if result[emotion].iloc[0].any():
                            try:
                                list_emotion_counts[idx]['emo_frq'][word_to_ind[word]] += 1
                            except:
                                continue
                else:
                    doc = nlp(word)
                    word_stemmed1 = " ".join([token.lemma_ for token in doc]).lower()
                    result1 = ncr[ncr['English (en)'] == word_stemmed1]
                    if not result1.empty:
                        # update the tweet-emotions counts
                        for idx, emotion in enumerate(emotions):
                            df_emotions.at[i, emotion] += result1[emotion].iloc[0]
                            # update the frequencies dictionary list
                            if result1[emotion].iloc[0].any():
                                try:
                                    list_emotion_counts[idx]['emo_frq'][word_to_ind[word_stemmed1]] += 1
                                except:
                                    continue
    df_tweets = pd.concat([df_tweets, df_emotions], axis=1)
    df_tweets.to_csv(output_path)

In [4]:
# from concurrent.futures import ProcessPoolExecutor, as_completed

# if __name__ == '__main__':

#     with ProcessPoolExecutor(max_workers=2) as executor:

        
#         futures = []
#         futures.append(executor.submit(analysis_to_csv, r"D:\covid research\output", r"D:\covid research\output\step3test.csv"))
#         futures.append(executor.submit(analysis_to_csv, r"D:\covid research\output", r"D:\covid research\output\step3test.csv"))
#         for future in as_completed(futures):
#             print(future.result())

analysis_to_csv(r"D:\covid research\test_set\s3 test input", r"D:\covid research\test_set\s3 test output\step3test.csv")

ValueError: No objects to concatenate